In [2]:
# ==========================================================
# High-Fidelity Artwork Analyzer v7 (Fully Local, Academic Tone, Refined Visuals)
# ==========================================================
# Metrics (same fidelity as v6):
#   • CIELAB color clustering
#   • Laplacian variance + Gabor texture (normalized)
#   • FFT spectral-residual saliency + focal centroid
#   • Hough lines + edge balance + dominant orientation
#   • Structural contour descriptors + illumination stats
#
# Visual upgrades:
#   • Translucent color fills (soft edges via blur)
#   • Amber/red texture "energy" field (gradient heat)
#   • Smooth saliency glow (soft colormap + alpha)
#   • Composition lines with subtle glow & orientation color
#   • Focal point with halo; subject box with drop-shadow
#   • Thoughtful additive blending to keep the artwork visible
#
# Outputs:
#   • overlay_color.png, overlay_texture.png, overlay_saliency.png,
#     overlay_composition.png, overlay_composite.png
#   • curatorial_summary.txt
# ==========================================================

import cv2
import numpy as np
from PIL import Image
import os

# ---------------------------
# Utility
# ---------------------------
def load_rgb(image_path):
    img = Image.open(image_path).convert("RGB")
    return np.array(img)

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def to_uint8(img_f32):
    img = np.clip(img_f32 * 255.0, 0, 255).astype(np.uint8)
    return img

def add_glow(base, center, color_bgr, radius=10, strength=0.8, layers=3):
    """Draw a soft glow (halo) at a point onto base (BGR float, 0..1)."""
    x, y = center
    h, w = base.shape[:2]
    overlay = base.copy()
    for i in range(layers, 0, -1):
        r = int(radius * (i * 1.0))
        alpha = (strength / layers) * (i / layers)
        tmp = overlay.copy()
        cv2.circle(tmp, (int(x), int(y)), r, (color_bgr[0], color_bgr[1], color_bgr[2]), -1, cv2.LINE_AA)
        overlay = cv2.addWeighted(tmp, alpha, overlay, 1 - alpha, 0)
    return overlay

def add_rect_shadow(base, rect, color_bgr, thickness=2, shadow=3):
    """Draw rectangle with soft drop shadow to improve readability."""
    x, y, w, h = rect
    # shadow
    for s in range(shadow, 0, -1):
        cv2.rectangle(base, (x+s, y+s), (x+w+s, y+h+s), (0,0,0), thickness, cv2.LINE_AA)
    # main
    cv2.rectangle(base, (x, y), (x+w, y+h), color_bgr, thickness, cv2.LINE_AA)
    return base

# ==========================================================
# 1) Color Analysis (CIELAB clustering)
# ==========================================================
def analyze_color_lab(img_rgb, k=6):
    H, W = img_rgb.shape[:2]
    lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    Z = lab.reshape((-1, 3)).astype(np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 0.3)
    _, labels, centers = cv2.kmeans(Z, k, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
    labels_2d = labels.reshape(H, W)
    counts = np.bincount(labels.flatten())
    order = np.argsort(-counts)
    centers = centers[order]
    centers_lab = centers.astype(np.uint8).reshape((-1, 1, 3))
    centers_rgb = cv2.cvtColor(centers_lab, cv2.COLOR_Lab2RGB).reshape((-1, 3)).astype(int)
    color_var = float(np.std(centers, axis=0).mean())
    return {
        "palette_rgb": [tuple(map(int, c)) for c in centers_rgb],
        "labels": labels_2d,
        "color_variability": color_var
    }

# ==========================================================
# 2) Texture (Laplacian + Gabor normalized)
# ==========================================================
def analyze_texture(img_rgb):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    gray_8u = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Laplacian variance (micro-contrast)
    lap = cv2.Laplacian(gray_8u, cv2.CV_64F, ksize=3)
    lap_var = float(lap.var())

    # Gabor filters (normalized by area to keep values human-scaled)
    orientations = [0, 30, 60, 90, 120, 150]
    wavelength = max(4, min(gray.shape)//40)
    H, W = gray_8u.shape
    gabor_energies = []
    for theta_deg in orientations:
        theta = np.deg2rad(theta_deg)
        kern = cv2.getGaborKernel((31, 31), 6.0, theta, wavelength, 0.5, 0)
        resp = cv2.filter2D(gray_8u, cv2.CV_32F, kern)
        # normalized per-pixel energy
        energy = float(np.mean(resp**2)) / (H * W / 1e5)
        gabor_energies.append(energy)
    gabor_mean = float(np.mean(gabor_energies))
    gabor_anisotropy = float(np.std(gabor_energies))

    return {
        "laplacian_variance": lap_var,
        "gabor_mean_energy": gabor_mean,
        "gabor_anisotropy": gabor_anisotropy
    }

# ==========================================================
# 3) Saliency (FFT spectral residual)
# ==========================================================
def spectral_residual_saliency(gray_f32):
    eps = 1e-8
    fft = np.fft.fft2(gray_f32)
    log_amp = np.log(np.abs(fft) + eps)
    phase = np.angle(fft)
    avg_log_amp = cv2.blur(log_amp, (3,3))
    spectral_residual = log_amp - avg_log_amp
    saliency = np.abs(np.fft.ifft2(np.exp(spectral_residual + 1j*phase)))**2
    saliency = cv2.GaussianBlur(saliency, (9,9), 2.0)
    sal_norm = (saliency - saliency.min()) / (saliency.max() - saliency.min() + eps)
    return (sal_norm * 255).astype(np.uint8)

def analyze_saliency(img_rgb):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY).astype(np.float32)
    sal = spectral_residual_saliency(gray)
    _, th = cv2.threshold(sal, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    H, W = gray.shape
    if contours:
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    else:
        x, y, w, h = W//4, H//4, W//2, H//2
    Y, X = np.indices(gray.shape)
    wts = sal.astype(np.float32) + 1
    cx = int((X*wts).sum() / wts.sum())
    cy = int((Y*wts).sum() / wts.sum())
    return {"saliency_map": sal, "subject_bbox": (x,y,w,h), "focal_point": (cx,cy)}

# ==========================================================
# 4) Composition (edges + Hough + orientation)
# ==========================================================
def analyze_composition(img_rgb):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    linesP = cv2.HoughLinesP(edges, 1, np.pi/180, 120, minLineLength=min(gray.shape)//8, maxLineGap=20)
    lines = []
    if linesP is not None:
        for l in linesP[:,0,:]:
            lines.append(tuple(int(v) for v in l))
    gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1)
    angles = (np.rad2deg(np.arctan2(gy, gx)) + 180) % 180
    mag = np.sqrt(gx**2 + gy**2)
    hist, bins = np.histogram(angles.flatten(), bins=12, range=(0,180), weights=mag.flatten())
    dominant_angle = float(0.5*(bins[np.argmax(hist)] + bins[np.argmax(hist)+1]))
    left_w = edges[:, :gray.shape[1]//2].sum()
    right_w = edges[:, gray.shape[1]//2:].sum()
    balance = (right_w - left_w)/(left_w + right_w + 1e-6)
    return {"edges": edges, "lines": lines, "edge_balance": balance, "dominant_angle_deg": dominant_angle}

# ==========================================================
# 5) Structural & Illumination
# ==========================================================
def analyze_structure(img_rgb):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 25, 5)
    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return {"area_ratio": 0, "aspect_ratio": 1, "eccentricity": 0}
    c = max(contours, key=cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    area_ratio = cv2.contourArea(c)/(gray.shape[0]*gray.shape[1])
    aspect_ratio = w/(h+1e-6)
    ecc = 0
    if len(c) >= 5:
        (cx,cy),(MA,ma),_ = cv2.fitEllipse(c)
        a,b = max(MA,ma)/2, min(MA,ma)/2
        ecc = np.sqrt(max(0, 1-(b*b)/(a*a)))
    return {"area_ratio": area_ratio, "aspect_ratio": aspect_ratio, "eccentricity": ecc}

def analyze_illumination(img_rgb):
    lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    L = lab[:,:,0].astype(np.float32)
    mu, sigma = L.mean(), L.std() + 1e-6
    skew = (((L - mu)/sigma)**3).mean()
    local = cv2.GaussianBlur(L, (0,0), 3)
    local_var = ((L - local)**2).mean()
    return {"lightness_skew": float(skew), "lightness_local_var": float(local_var)}

# ==========================================================
# 6) Report (academic tone)
# ==========================================================
def academic_report(metrics):
    color = metrics["color"]; texture = metrics["texture"]; comp = metrics["composition"]
    sal = metrics["saliency"]; struct = metrics["structure"]; illum = metrics["illumination"]

    avg = tuple(np.array(color["palette_rgb"]).mean(axis=0).astype(int))
    temp = "warm" if avg[0] > avg[2] + 10 else "cool" if avg[2] > avg[0] + 10 else "neutral"
    lapv, ganiso = texture["laplacian_variance"], texture["gabor_anisotropy"]
    n_lines, bal = len(comp["lines"]), comp["edge_balance"]
    area, aspect, ecc = struct["area_ratio"], struct["aspect_ratio"], struct["eccentricity"]
    skew = illum["lightness_skew"]
    color_var = color["color_variability"]

    score = (lapv*0.25) + (ganiso*20*0.15) + (n_lines*0.1) + (abs(bal)*40*0.1) + (color_var*0.25) + (abs(skew)*8*0.15)
    if score < 18: style = "Minimalist"
    elif score < 28: style = "Realist"
    elif score < 40: style = "Impressionist"
    elif score < 55: style = "Expressionist"
    elif score < 70: style = "Surrealist"
    else: style = "Abstract-Contemporary"

    micro = (f"{style} tendencies; {temp} palette; {n_lines} structural lines; "
             f"surface micro-contrast {lapv:.1f}.")
    deep = (
        f"The work pursues a {temp} chromatic register with palette variability {color_var:.1f} "
        f"(mean tone approx. RGB {avg}). The surface exhibits micro-contrast at {lapv:.1f} "
        f"and directional emphasis (Gabor anisotropy {ganiso:.2f}). The composition "
        f"articulates ≈{n_lines} linear trajectories (edge balance {bal:.2f}), organized around a focal "
        f"at {sal['focal_point']}. Structural measures (area ratio {area:.3f}, aspect {aspect:.2f}, "
        f"eccentricity {ecc:.2f}) and a lightness skew of {skew:.2f} inform the work’s optical behavior."
    )
    return style, micro, deep

# ==========================================================
# 7) Aesthetic Overlays
# ==========================================================
def make_color_overlay(img_rgb, palette_rgb, blur_px=9, alpha=0.35):
    lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    H, W = lab.shape[:2]
    color_accum = np.zeros_like(img_rgb, dtype=np.float32)

    for c in palette_rgb:
        patch = np.uint8([[c]])
        lab_patch = cv2.cvtColor(patch, cv2.COLOR_RGB2LAB)[0][0]
        lower = np.clip(lab_patch - np.array([20,15,15]), 0, 255)
        upper = np.clip(lab_patch + np.array([20,15,15]), 0, 255)
        mask = cv2.inRange(lab, lower, upper)
        # soften mask
        mask = cv2.GaussianBlur(mask.astype(np.float32)/255.0, (0,0), blur_px)
        color_accum += np.dstack([mask, mask, mask]) * (np.array(c, dtype=np.float32)/255.0)

    # Normalize and apply alpha
    color_accum = np.clip(color_accum, 0, 1)
    return (color_accum * alpha).astype(np.float32)

def make_texture_overlay(img_rgb, heat_color1=(1.0,0.6,0.0), heat_color2=(1.0,0.1,0.0), top_pct=98, blur_px=6, alpha=0.35):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    lap = np.abs(cv2.Laplacian(gray, cv2.CV_32F))
    lap_norm = cv2.normalize(lap, None, 0, 1, cv2.NORM_MINMAX)
    thr = np.percentile(lap_norm, top_pct)
    mask = (lap_norm >= thr).astype(np.float32)
    mask = cv2.GaussianBlur(mask, (0,0), blur_px)
    # gradient from heat_color1 to heat_color2 by strength
    heat = np.zeros_like(img_rgb, dtype=np.float32)
    for i in range(3):
        heat[:,:,i] = (heat_color1[i] * (1 - lap_norm) + heat_color2[i] * lap_norm) * mask
    return (heat * alpha).astype(np.float32)

def make_saliency_overlay(saliency_map, alpha=0.25, blur_px=3):
    sal = cv2.GaussianBlur(saliency_map, (0,0), blur_px)
    sal = cv2.normalize(sal.astype(np.float32), None, 0, 1, cv2.NORM_MINMAX)
    # custom smooth colormap: dark red -> orange -> yellow
    r = np.clip(0.2 + 0.8*sal, 0, 1)
    g = np.clip(0.0 + 0.8*sal, 0, 1)
    b = np.clip(0.0 + 0.2*sal, 0, 1)
    sal_rgb = np.dstack([r,g,b])
    return (sal_rgb * alpha).astype(np.float32)

def make_composition_overlay(img_rgb, lines, alpha=0.7, glow=0.6, keep_top_percent=10):
    overlay = np.zeros_like(img_rgb, dtype=np.float32)
    if not lines:
        return overlay

    lengths = [np.hypot(x2-x1, y2-y1) for (x1,y1,x2,y2) in lines]
    cutoff = np.percentile(lengths, 100 - keep_top_percent)

    # Base lines
    for (x1,y1,x2,y2), L in zip(lines, lengths):
        if L < cutoff: 
            continue
        angle = (np.rad2deg(np.arctan2(y2-y1, x2-x1)) + 180) % 180
        if angle < 30 or angle > 150:
            color = (1.0, 0.2, 0.2)  # vertical-ish → red-ish
        elif 60 < angle < 120:
            color = (0.2, 1.0, 0.2)  # horizontal-ish → green-ish
        else:
            color = (0.2, 1.0, 1.0)  # diagonal → cyan-ish
        cv2.line(overlay, (x1,y1), (x2,y2), (color[2], color[1], color[0]), 2, cv2.LINE_AA)

    overlay = cv2.GaussianBlur(overlay, (0,0), 1.2)

    # Glow layer
    glow_layer = cv2.GaussianBlur(overlay, (0,0), 3.0)
    comp = cv2.addWeighted(glow_layer, glow, overlay, 1 - glow, 0)
    return (comp * alpha).astype(np.float32)

# # ==========================================================
# 8) Unified Composite Visualization (single image output)
# ==========================================================
def visualize_and_export(img_rgb, metrics, out_dir):
    ensure_dir(out_dir)
    H, W = img_rgb.shape[:2]
    base = img_rgb.astype(np.float32) / 255.0

    # === Overlays ===
    color_overlay = make_color_overlay(img_rgb, metrics["color"]["palette_rgb"], blur_px=9, alpha=0.35)
    texture_overlay = make_texture_overlay(img_rgb, alpha=0.35)
    saliency_overlay = make_saliency_overlay(metrics["saliency"]["saliency_map"], alpha=0.25, blur_px=3)
    comp_overlay = make_composition_overlay(img_rgb, metrics["composition"]["lines"], alpha=0.65, glow=0.55)

    # === Composite (balanced additive blend) ===
    composite = (
        0.70 * base +               # keep artwork visible
        0.30 * color_overlay +
        0.30 * texture_overlay +
        0.25 * saliency_overlay +
        0.40 * comp_overlay
    )
    composite = np.clip(composite, 0, 1)

    # === Focal halo + subject box ===
    fx, fy = metrics["saliency"]["focal_point"]
    comp_bgr = composite[:, :, ::-1].copy()
    comp_bgr = add_glow(comp_bgr, (fx, fy), color_bgr=(0, 0, 255), radius=12, strength=0.9, layers=4)
    sx, sy, sw, sh = metrics["saliency"]["subject_bbox"]
    comp_bgr = add_rect_shadow(comp_bgr, (sx, sy, sw, sh), color_bgr=(255, 0, 255), thickness=2, shadow=3)
    composite = comp_bgr[:, :, ::-1]

    # === Add label legend ===
    legend = np.full((80, W, 3), 255, dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(legend, "Legend:", (20, 50), font, 0.8, (0,0,0), 2)
    cv2.putText(legend, "Red Halo = Focal point   Magenta Box = Subject region   Green/Cyan/Red = Composition lines", 
                (150, 50), font, 0.6, (60,60,60), 1, cv2.LINE_AA)
    composite_uint8 = to_uint8(composite)
    final_img = np.vstack([composite_uint8, legend])

    # === Save only one image ===
    out_path = os.path.join(out_dir, "overlay_final_composite.png")
    cv2.imwrite(out_path, final_img)
    print(f"✅ Single composite overlay saved to: {out_path}")


# ==========================================================
# 9) Master routine
# ==========================================================
def analyze_artwork_high_fidelity(image_path):
    img_rgb = load_rgb(image_path)

    metrics = {
        "color": analyze_color_lab(img_rgb),
        "texture": analyze_texture(img_rgb),
        "saliency": analyze_saliency(img_rgb),
        "composition": analyze_composition(img_rgb),
        "structure": analyze_structure(img_rgb),
        "illumination": analyze_illumination(img_rgb)
    }

    style, micro, deep = academic_report(metrics)
    print("\n========== CURATORIAL SUMMARY ==========")
    print("Style (inferred):", style)
    print("\n— Micro —\n", micro)
    print("\n— Deep —\n", deep)

    out_dir = os.path.splitext(image_path)[0] + "_analysis"
    visualize_and_export(img_rgb, metrics, out_dir)
    with open(os.path.join(out_dir, "curatorial_summary.txt"), "w") as f:
        f.write(f"Style: {style}\n\nMicro:\n{micro}\n\nDeep:\n{deep}")
    print(f"\n✅ Saved overlays and summary in: {out_dir}")

# ==========================================================
# Run
# ==========================================================
if __name__ == "__main__":
    # 👇 Set your artwork image path here
    IMAGE_PATH = "/Users/alievanayasso/Documents/SlowMA/Modigliani.jpg"
    analyze_artwork_high_fidelity(IMAGE_PATH)



========== CURATORIAL SUMMARY ==========
Style (inferred): Abstract-Contemporary

— Micro —
 Abstract-Contemporary tendencies; warm palette; 366 structural lines; surface micro-contrast 7064.8.

— Deep —
 The work pursues a warm chromatic register with palette variability 32.9 (mean tone approx. RGB (np.int64(157), np.int64(141), np.int64(94))). The surface exhibits micro-contrast at 7064.8 and directional emphasis (Gabor anisotropy 1033709.84). The composition articulates ≈366 linear trajectories (edge balance 0.25), organized around a focal at (1648, 1857). Structural measures (area ratio 0.999, aspect 0.81, eccentricity 0.00) and a lightness skew of -1.05 inform the work’s optical behavior.
✅ Single composite overlay saved to: /Users/alievanayasso/Documents/SlowMA/Modigliani_analysis/overlay_final_composite.png

✅ Saved overlays and summary in: /Users/alievanayasso/Documents/SlowMA/Modigliani_analysis
